In [3]:
# Required Dependencies:

# pip install chromadb
# pip install langchain_groq
# pip install langchain_community

In [73]:
# Importing required modules
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import csv
import uuid
import chromadb

In [7]:
# Extracts the Groq API Key from '.env.local' file

import os                                                                                                                                                                                                          
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path(".env.local"))
KEY = os.getenv("GROQ_API_KEY")

## Testing the ChatGroq

In [10]:
# Testing if ChatGroq is running correctly

llm = ChatGroq(
    model= 'llama-3.3-70b-versatile',
    temperature= 0,
    groq_api_key= KEY,
    max_tokens= None,
    timeout= None,
    max_retries= 2,
    # Other Parameters...
)

answer = llm.invoke("Who is the richest man in the world of all time?")
print(answer.content)
print("\n ***Preambled Answer***\n")
answer = llm.invoke("Who is the richest man in the world of all time?, no preamble")
print(answer.content)

The richest man in the world of all time is a matter of debate among historians and economists, as the concept of wealth and its measurement have changed over time. However, according to various sources, including Forbes and other financial publications, the richest person in history is often considered to be:

1. **Mansa Musa I** (1280-1337): The king of the Mali Empire, who ruled over a vast territory in West Africa during the 14th century. His wealth is estimated to be around $400 billion in today's dollars, making him the richest person in history.

Mansa Musa's wealth came from his control of the trans-Saharan trade, which included gold, salt, and other valuable commodities. He was known for his extravagant spending and generosity, and his pilgrimage to Mecca in 1324 was said to have been so lavish that it caused a shortage of gold in Egypt.

Other contenders for the title of richest person in history include:

* **John D. Rockefeller** (1839-1937): The American oil industry magna

In [12]:
# Chromadb Setup

client = chromadb.Client()
collection = client.create_collection(name = "my_collection")

In [13]:
collection.add(
    
    documents = [
        "I like drinking water everyday",
        "I like my apple iphone",
        "Do you like apple?"
    ],

    ids = ["id1", "id2", "id3"],

    metadatas = [
        {"url": "source_to_id1"},
        {"url": "source_to_id2"},
        {"url": "source_to_id3"},
    ]
)

In [14]:
print(collection.get())

{'ids': ['id1', 'id2', 'id3'], 'embeddings': None, 'documents': ['I like drinking water everyday', 'I like my apple iphone', 'Do you like apple?'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [{'url': 'source_to_id1'}, {'url': 'source_to_id2'}, {'url': 'source_to_id3'}]}


In [18]:
print(collection.get(ids = ["id2"]))

{'ids': ['id2'], 'embeddings': None, 'documents': ['I like my apple iphone'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [{'url': 'source_to_id2'}]}


In [20]:
collection.query(query_texts = ["I like technology"], n_results = 2)

{'ids': [['id2', 'id3']],
 'embeddings': None,
 'documents': [['I like my apple iphone', 'Do you like apple?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'url': 'source_to_id2'}, {'url': 'source_to_id3'}]],
 'distances': [[1.0667046308517456, 1.2515640258789062]]}

In [22]:
collection.query(query_texts = ["Do you eat healthy?"], n_results = 1)

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['I like drinking water everyday']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'url': 'source_to_id1'}]],
 'distances': [[1.3786277770996094]]}

## Extracting Data From Job Description

In [25]:
# Using the WebBaseLoader to scrape content of a webpage. Using a demo job description link for testing purposes
loader = WebBaseLoader("https://jobs.foundever.com/job/Canada-IT-Site-System-Supt-Admin-Onsite-in-London%2C-ON-Cana/1286688600/?")

# Stores the loaded data of the webpage
data = loader.load().pop().page_content

In [154]:
# Defining the LLM that is going to be used for data extraction

llm = ChatGroq(
    model= 'llama-3.3-70b-versatile',
    temperature= 0.5,
    groq_api_key= KEY,
    max_tokens= None,
    timeout= None,
    max_retries= 2,
    # Other Parameters...
)

In [156]:
# Using prompt on the model

extract_from_prompt = PromptTemplate.from_template(
    """
    I will give you scraped text from job postings. Your job is to extract the details and requirements in a JSON format containing the following keywords: 'role', 'experience', 'skills', 'description'
    Only return the JSON. No preamble please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = extract_from_prompt | llm
answer = chain_extract.invoke(input= {'page_data': data})
print(answer.content)

```json
{
    "role": "Data Engineer/Scientist",
    "experience": "Not specified",
    "skills": [
        "Python",
        "SQL",
        "Pandas",
        "Airflow",
        "PySpark",
        "Spark SQL",
        "Delta Lake",
        "Machine Learning",
        "Deep Learning",
        "TensorFlow",
        "Data Engineering",
        "ETL",
        "ELT",
        "Cloud Platforms (AWS, GCP, Azure)",
        "Data Warehousing",
        "Data Modeling",
        "DBT",
        "Data Visualization",
        "Power BI",
        "Tableau",
        "MLOps",
        "MLflow",
        "Kubeflow",
        "Natural Language Processing (NLP)",
        "NLTK",
        "spaCy",
        "Computer Vision",
        "OpenCV",
        "Time Series Analysis",
        "Forecasting",
        "Prophet",
        "Data Cleaning",
        "Data Wrangling",
        "Feature Engineering",
        "Scikit-learn",
        "Statistical Analysis",
        "Hypothesis Testing",
        "Data Ethics",
        "P

In [157]:
# Converting the output into JSON using parser

parser = JsonOutputParser()
json_answer = parser.parse(answer.content)
json_answer

{'role': 'Data Engineer/Scientist',
 'experience': 'Not specified',
 'skills': ['Python',
  'SQL',
  'Pandas',
  'Airflow',
  'PySpark',
  'Spark SQL',
  'Delta Lake',
  'Machine Learning',
  'Deep Learning',
  'TensorFlow',
  'Data Engineering',
  'ETL',
  'ELT',
  'Cloud Platforms (AWS, GCP, Azure)',
  'Data Warehousing',
  'Data Modeling',
  'DBT',
  'Data Visualization',
  'Power BI',
  'Tableau',
  'MLOps',
  'MLflow',
  'Kubeflow',
  'Natural Language Processing (NLP)',
  'NLTK',
  'spaCy',
  'Computer Vision',
  'OpenCV',
  'Time Series Analysis',
  'Forecasting',
  'Prophet',
  'Data Cleaning',
  'Data Wrangling',
  'Feature Engineering',
  'Scikit-learn',
  'Statistical Analysis',
  'Hypothesis Testing',
  'Data Ethics',
  'Privacy',
  'GDPR',
  'Big Data',
  'Hadoop',
  'Spark',
  'Data Governance',
  'Data Quality',
  'Metadata Management',
  'Data Security',
  'Encryption',
  'Tokenization',
  'Cloud Data Engineering',
  'AWS Glue',
  'Databricks'],
 'description': 'The rol

In [160]:
# Checking the type of output for confirmation
print(type(json_answer))

<class 'dict'>


#### Importing Portfolio

In [205]:
# Function reads and formats the data in portfolio so that the LLM can understand

def read_file(path):
    current = []
    with open(path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)
        for line in csv_reader:
            # Formatting data
            skills = tuple(line[:-1])
            link = line[-1]
            current.append((skills, link))
        return current

In [207]:
# Sample Usage

path = 'demo_portfolio.csv'
data = read_file(path)


for skills, link in data:
    print(skills, link)

('Python', ' SQL', ' Pandas')  https://github.com/user/project1
('SQL', ' Python', ' Airflow')  https://github.com/user/project2
('PySpark', ' Spark SQL', ' Delta Lake')  https://github.com/user/project3
('Machine Learning', ' Deep Learning', ' TensorFlow')  https://github.com/user/project4
('Data Engineering', ' ETL', ' ELT')  https://github.com/user/project5
('Cloud Platforms (AWS', ' GCP', ' Azure)')  https://github.com/user/project6
('Data Warehousing', ' Data Modeling', ' DBT')  https://github.com/user/project7
('Data Visualization', ' Power BI', ' Tableau')  https://github.com/user/project8
('MLOps', ' MLflow', ' Kubeflow')  https://github.com/user/project9
('Natural Language Processing (NLP)', ' NLTK', ' spaCy')  https://github.com/user/project10
('Computer Vision', ' OpenCV', ' TensorFlow')  https://github.com/user/project11
('Time Series Analysis', ' Forecasting', ' Prophet')  https://github.com/user/project12
('Data Cleaning', ' Data Wrangling', ' Pandas')  https://github.com

#### Inserting data into Vector Database

In [210]:
# Chromadb Setup

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name= 'portfolio_links')

if not collection.count():
    for skills, links in data:
        collection.add(
            documents= str(skills),
            metadatas= {'portfolio_url': links},
            ids= [str(uuid.uuid4())]
        )

In [212]:
# Displays the first skills required in the sample job description

json_answer['skills'][0]

'Python'

In [214]:
# Queries the portfolio for a relevant project based on the first required skill as above

urls = collection.query(query_texts= json_answer['skills'][0], n_results= 2)
urls

{'ids': [['e4e3821b-db9a-4430-8114-1eb7453ee5f8',
   'ef127efc-1cf1-41ce-beab-937f122ff417']],
 'embeddings': None,
 'documents': [["('SQL', ' Python', ' Airflow')",
   "('Python', ' SQL', ' Pandas')"]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'portfolio_url': ' https://github.com/user/project2'},
   {'portfolio_url': ' https://github.com/user/project1'}]],
 'distances': [[1.009972095489502, 1.0451734066009521]]}

## Setting up prompt for writing the email

In [221]:
# Assigning variable for the dob description in JSON

job_desc = json_answer['description']

# Prompt for the email

email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your role: Your name is Fahim. You are an incredible business development officer who who knows how to get clients. You work for ABC Consulting Firm, your firm works with all kinds of IT clients and provide solutions in the domain of Data Science and AI.
    CoverLetter AI focuses on efficient tailored solutions for all clients keeping costs down. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

# Extracting the answer
chain_email = email_prompt | llm
answer = chain_email.invoke({'job_description': job_desc, 'portfolio_urls': urls})

In [222]:
# Email that has been generated

email = answer.content
print(email)

Subject: Expert Data Science and AI Solutions for Your Engineering and Science Projects

Dear Hiring Manager,

I came across your job posting for a data engineering and science role, and I was impressed by the scope of the projects you're working on, including data warehousing, data modeling, data visualization, machine learning, and data governance. As a Business Development Officer at ABC Consulting Firm, I'd like to introduce you to our team of experts who can help you achieve your goals efficiently and cost-effectively.

At ABC Consulting Firm, we specialize in providing tailored Data Science and AI solutions that cater to the unique needs of our clients. Our team has extensive experience working with a range of tools and technologies, including Python, SQL, Pandas, Airflow, PySpark, and TensorFlow. We're confident that our expertise aligns perfectly with your project requirements.

I'd like to draw your attention to some of our notable projects that demonstrate our capabilities:

